<a href="https://colab.research.google.com/github/duatama2004/Komputasi_pararel/blob/main/Duatama%20Munif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
# -*- coding: utf-8 -*-
"""
Improved Flask application with Ngrok integration for Google Colab or local use.
"""

# Install dependencies
!pip install flask pyngrok tensorflow

# Import required libraries
from flask import Flask, request, render_template, jsonify
import tensorflow as tf
import numpy as np
from PIL import Image
import os
from pyngrok import ngrok


In [79]:

# Path to model stored on Google Drive
MODEL_PATH = "/content/drive/MyDrive/Komputasi pararel UAS duatama/Datasets/full_model.h5"
TEMPLATE_FOLDER = "/content/drive/MyDrive/Komputasi pararel UAS duatama/Datasets/templates"
STATIC_FOLDER = "/content/drive/MyDrive/Komputasi pararel UAS duatama/Datasets/static"


# Initialize Flask app
app = Flask(__name__, template_folder=TEMPLATE_FOLDER, static_folder=STATIC_FOLDER)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
# Load the CNN model
def load_model():
    try:
        model = tf.keras.models.load_model(MODEL_PATH)
        print("Model loaded successfully.")
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

model = load_model()

Model loaded successfully.


In [81]:
# Function to predict image class
def predict_image(image_path):
    if model is None:
        raise ValueError("Model is not loaded.")

    try:
        img = Image.open(image_path).resize((100, 100))  # Adjust input size as needed
        img_array = np.array(img) / 255.0  # Normalize pixels
        img_array = np.expand_dims(img_array, axis=0)

        predictions = model.predict(img_array)
        class_idx = np.argmax(predictions)
        confidence = np.max(predictions)
        return class_idx, confidence
    except Exception as e:
        raise ValueError(f"Prediction error: {e}")

In [82]:
# Flask route: Home
@app.route("/", methods=["GET"])
def home():
    try:
        return render_template("index.html")
    except Exception as e:
        return f"Error rendering template: {e}", 500

# Flask route: Predict endpoint
@app.route("/predict", methods=["POST"])
def predict():
    if 'file' not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files['file']
    os.makedirs("temp", exist_ok=True)
    file_path = os.path.join("temp", file.filename)
    file.save(file_path)

    try:
        class_idx, confidence = predict_image(file_path)
        os.remove(file_path)
        return render_template("result.html", class_index=class_idx, confidence=confidence * 100)
    except Exception as e:
        os.remove(file_path)
        print(f"Error during prediction: {e}")
        return jsonify({"error": str(e)}), 500


In [83]:
!ngrok authtoken '2qv83NpOcSOFRLN1qkfk0TIBrQt_5TqVrK3cPGgFwz4XDGBV'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [84]:
# Ngrok setup and app run
if __name__ == "__main__":
    try:
        public_url = ngrok.connect(5000)
        print(f"Ngrok public URL: {public_url}")

        app.run(host="0.0.0.0", port=5000)
    except Exception as e:
        print(f"Error starting app: {e}")

Ngrok public URL: NgrokTunnel: "https://bb1b-34-23-224-30.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Jan/2025 12:09:25] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2025 12:09:26] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Jan/2025 12:09:27] "GET /favicon.ico HTTP/1.1" 404 -
